In [33]:
import numpy as np
from tqdm import tqdm
import re
import pickle
from multiprocessing import Pool
from tqdm import tqdm
from rdkit import Chem
from rdkit import RDLogger
from rdkit.Chem import Draw
from multiprocessing import Pool
import pandas as pd

RDLogger.DisableLog('rdApp.*')

In [2]:
NUM_LEN = 100

In [3]:
with open("./data/CID-SMILES","r") as f:
    lines = [ f.readline() for _ in tqdm(range(NUM_LEN)) ]
    #f.readlines()
    list_smiles = list( map(lambda line: re.split("\s+", line.strip() )[-1], lines ) )
    
np.random.seed(1)
np.random.shuffle(list_smiles)

100%|██████████| 100/100 [00:00<00:00, 103384.37it/s]


In [4]:
list_smiles

['CC(C)(COP(=O)(O)OP(=O)(O)OCC1C(C(C(O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)(O)O)C(C(=O)NCCC(=O)NCCSC(=O)CC(C)(CC(=O)O)O)O',
 'CC(CO)C(=O)SCCNC(=O)CCNC(=O)C(C(C)(C)COP(=O)(O)OP(=O)(O)OCC1C(C(C(O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)(O)O)O',
 'C(C(C(CO)O)O)C(=O)C(=O)O',
 'C[N+](C)(C)CC(CC(=O)O)O',
 'C(C1C(C(C(C(O1)O)O)O)OC2C(C(=O)C(C(O2)CO)O)O)O',
 'C1=CC(=C(C(=C1)O)O)C(=O)O',
 'C1C(C(C(OC1O)CO)O)O',
 'C1=CC(=C(C(=C1)O)N)C(=O)O',
 'C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)COP(=O)(O)O)OP(=O)(O)O)O)N',
 'CC(=O)C=CC=C(C(=O)O)O',
 'CC(=O)CC(=O)O',
 'C(CC(=O)O)C(C(=O)O)O',
 'CC(CO)(C(COP(=O)(O)OP(=O)(O)OCC1C(C(C(O1)N2C=CC(=NC2=O)N)O)O)O)OP(=O)(O)O',
 'CC=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCC1=C(C(=CC=C1)OC)O)C)C)C)C)C)C)C',
 'CC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCC1=C(C(=CC=C1)O)O)C)C)C)C)C)C)C)C',
 'CC(C)C(C(C(=O)O)O)C(=O)O',
 'C1=CC(OC1=O)(CC(=O)O)Cl',
 'C(CC(=O)N)C(=O)C(=O)O',
 'C=C(C(=O)O)OC1CC(=CC(C1O)OP(=O)(O)O)C(=O)O',
 'C1=C(C=C(C(=C1O)O)O)O',
 'C1=CC(C(C(=C1)C(=O)O)O)O',
 'CC(C)C(CC(=O)

In [5]:
list_smiles[0]

'CC(C)(COP(=O)(O)OP(=O)(O)OCC1C(C(C(O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)(O)O)C(C(=O)NCCC(=O)NCCSC(=O)CC(C)(CC(=O)O)O)O'

In [22]:
def check_image(smiles):
    if len(smiles) >= 150:
        return False

    m = Chem.MolFromSmiles(smiles)
    if (m!=None):
        try:
            img = Draw.MolToImage(m, size=(300,300))
            return True
        except Exception as e:
            print("1: ", smiles, e)
            return False
    return False

#def pool_filter(pool, func, candidates):
#    return [c for c, keep in zip(candidates, pool.imap(func, tqdm(candidates) )) if keep]

In [21]:
#list_smiles = pool_filter(p, check_image, list_smiles)

In [34]:
raw_data = {'image_id' : [], 'InChI' : []}
data = pd.DataFrame(raw_data)
data

,image_id,InChI


In [39]:
def make_inchi(smile) : 
    m = Chem.MolFromSmiles(smile)
    inchi = Chem.rdinchi.MolToInchi(m)
    return inchi[0]

In [40]:
for i, smile in tqdm(enumerate(list_smiles)) :
    if check_image(smile) :
        inchi = make_inchi(smile)
        image_id = str(i).zfill(7)
        data.loc[i] = [image_id, inchi]

100it [00:00, 161.31it/s]


In [45]:
data

,image_id,InChI
0,0000000,"InChI=1S/C27H44N7O20P3S/c1-26(2,21(40)24(41)30..."
1,0000001,InChI=1S/C25H42N7O18P3S/c1-13(8-33)24(38)54-7-...
2,0000002,InChI=1S/C6H10O6/c7-2-5(10)3(8)1-4(9)6(11)12/h...
3,0000003,"InChI=1S/C7H15NO3/c1-8(2,3)5-6(9)4-7(10)11/h6,..."
4,0000004,InChI=1S/C12H20O11/c13-1-3-5(15)6(16)9(19)12(2...
...,...,...
95,0000095,"InChI=1S/C6H4N2/c7-4-6-2-1-3-8-5-6/h1-3,5H"
96,0000096,InChI=1S/C2H4Cl2/c3-1-2-4/h1-2H2
97,0000097,InChI=1S/C19H28O2/c1-18-9-7-13(20)11-12(18)3-4...
98,0000098,InChI=1S/C20H32O5/c1-2-3-6-9-15(21)12-13-17-16...


In [44]:
print(len(data))

100
